In [2]:
import pandas as pd
import os
import glob
from collections import Counter

directory_path = "./cleaned_data/"

csv_files = glob.glob(os.path.join(directory_path, "*.csv"))

origin_iata_counter = Counter()


for file_path in csv_files:
    try:
        file_name = os.path.basename(file_path)
        print(f"Reading: {file_name}")


        chunk_size = 500000

        for chunk in pd.read_csv(file_path, usecols=["ORIGIN_IATA"], chunksize=chunk_size):
            value_counts = chunk["ORIGIN_IATA"].value_counts().to_dict()
            origin_iata_counter.update(value_counts)

        print(f"{file_name} finished reading")

    except Exception as e:
        print(f"Error in reading {file_path}: {e}")

# 获取不同ORIGIN_IATA的总数
total_unique_iata = len(origin_iata_counter)
print(f"\n Find {total_unique_iata} distinct IATA codes")

# 获取出现次数最多的前100个机场
top_100_airports = origin_iata_counter.most_common(100)

# 打印结果
for i, (iata, count) in enumerate(top_100_airports, 1):
    print(f"{i}\t{iata}\t{count:,}")

result_df = pd.DataFrame(top_100_airports, columns=['ORIGIN_IATA', 'Times'])
result_df.index = result_df.index + 1
result_df.index.name = 'Rank'
result_df.to_csv("top_100_airports.csv")

print("\nSaved in top_100_airports.csv")

Reading: August2021.csv
August2021.csv finished reading
Reading: August2022.csv
August2022.csv finished reading
Reading: August2023.csv
August2023.csv finished reading
Reading: August2024.csv
August2024.csv finished reading
Reading: July2021.csv
July2021.csv finished reading
Reading: July2022.csv
July2022.csv finished reading
Reading: July2023.csv
July2023.csv finished reading
Reading: July2024.csv
July2024.csv finished reading
Reading: June2021.csv
June2021.csv finished reading
Reading: June2022.csv
June2022.csv finished reading
Reading: June2023.csv
June2023.csv finished reading
Reading: June2024.csv
June2024.csv finished reading
Reading: May2021.csv
May2021.csv finished reading
Reading: May2022.csv
May2022.csv finished reading
Reading: May2023.csv
May2023.csv finished reading
Reading: May2024.csv
May2024.csv finished reading

 Find 378 distinct IATA codes
1	ATL	457,121
2	ORD	429,876
3	DFW	414,896
4	DEN	411,220
5	CLT	323,043
6	LAX	270,765
7	SEA	261,944
8	LAS	241,505
9	PHX	231,414
10	

In [4]:
import pandas as pd
import os
import glob
import re
from datetime import datetime
import numpy as np

# Define the path to the weather data folder
weather_data_path = 'weather_data/'  # Replace with the actual path

# Define output directory
output_dir = './cleaned_weather_data/'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created output directory: {output_dir}")

# Get all CSV files
csv_files = glob.glob(os.path.join(weather_data_path, "*.csv"))

# Create a dictionary to organize files
airport_files = {}

# Parse filenames and organize files
for file_path in csv_files:
    file_name = os.path.basename(file_path)
    # Extract IATA code, year, and month from filename
    match = re.match(r'([A-Z]+)_(\d{4})_([A-Za-z]+)_(\d+)\.csv', file_name)
    if match:
        iata, year, month, station = match.groups()
        key = f"{iata}_{year}_{month}"

        if key not in airport_files:
            airport_files[key] = []

        airport_files[key].append(file_path)

# List of extreme weather columns to track
extreme_weather_cols = ['WT01', 'WT03', 'WT04', 'WT05', 'WT08', 'WT11']
# Add precipitation column
precipitation_col = 'PRCP'

# Process each airport-year-month combination
for key, file_list in airport_files.items():
    print(f"Processing {key}...")

    # Create empty DataFrame to store merged results
    result_df = None

    # Process all station files for this airport-year-month
    for file_path in file_list:
        try:
            # Read the CSV file
            df = pd.read_csv(file_path)

            # Ensure DATE column is in datetime format
            df['DATE'] = pd.to_datetime(df['DATE'])

            # Keep only date, extreme weather columns, and precipitation
            cols_to_keep = ['DATE'] + [col for col in extreme_weather_cols if col in df.columns]
            if precipitation_col in df.columns:
                cols_to_keep.append(precipitation_col)

            station_df = df[cols_to_keep].copy()

            # Handle missing values - replace NaN with 0 for extreme weather
            for col in extreme_weather_cols:
                if col in station_df.columns:
                    station_df[col] = station_df[col].fillna(0).astype(int)
                else:
                    station_df[col] = 0

            # Initialize result_df if it's empty
            if result_df is None:
                result_df = station_df
            else:
                # Merge data, taking maximum values for extreme weather (any station reporting counts as occurrence)
                result_df = pd.merge(result_df, station_df, on='DATE', how='outer', suffixes=('', f'_{file_path}'))

                # Process merged extreme weather columns
                for col in extreme_weather_cols:
                    # Find all columns related to this weather phenomenon
                    related_cols = [c for c in result_df.columns if c.startswith(col) and (c == col or c.startswith(f"{col}_"))]

                    if len(related_cols) > 1:
                        # Calculate if any station reported extreme weather
                        result_df[col] = result_df[related_cols].max(axis=1).fillna(0).astype(int)

                        # Remove temporary columns
                        for temp_col in related_cols:
                            if temp_col != col:
                                result_df = result_df.drop(columns=[temp_col])

        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    # If no valid data was found, continue to next key
    if result_df is None:
        print(f"Warning: No valid data for {key}")
        continue

    # Process precipitation column after all files are merged
    prcp_cols = [c for c in result_df.columns if c == precipitation_col or c.startswith(f"{precipitation_col}_")]

    if prcp_cols:
        # Custom function to calculate average precipitation
        # Only include positive values in the average calculation
        def avg_positive_prcp(row):
            # Get all precipitation values from the row
            values = [row[col] for col in prcp_cols]
            # Filter out NaN values
            values = [v for v in values if pd.notnull(v)]
            # Filter out zero values (only include positive precipitation)
            positive_values = [v for v in values if v > 0]

            # If there are positive values, return their average
            if positive_values:
                return sum(positive_values) / len(positive_values)
            # If there are valid measurements but all are zero, return 0
            elif values:
                return 0
            # If no valid data, return NaN
            else:
                return np.nan

        # Apply the custom function to calculate average precipitation
        result_df[precipitation_col] = result_df.apply(avg_positive_prcp, axis=1)

        # Remove temporary precipitation columns
        for temp_col in prcp_cols:
            if temp_col != precipitation_col:
                result_df = result_df.drop(columns=[temp_col])

    # Reorganize results
    # Keep only DATE, extreme weather columns, and precipitation
    final_cols = ['DATE'] + extreme_weather_cols
    if precipitation_col in result_df.columns:
        final_cols.append(precipitation_col)

    result_df = result_df[final_cols].copy()

    # Add an EXTREME_WEATHER column indicating presence of any extreme weather phenomenon
    result_df['EXTREME_WEATHER'] = result_df[extreme_weather_cols].max(axis=1)

    # Sort by date
    result_df = result_df.sort_values('DATE')

    # Save results to CSV file in the output directory
    output_file = os.path.join(output_dir, f"{key}.csv")
    result_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

print("Processing complete!")

Created output directory: ./cleaned_weather_data/
Processing ABI_2021_Aug...
Results saved to ./cleaned_weather_data/ABI_2021_Aug.csv
Processing ABI_2021_Jul...
Results saved to ./cleaned_weather_data/ABI_2021_Jul.csv
Processing ABI_2021_Jun...
Results saved to ./cleaned_weather_data/ABI_2021_Jun.csv
Processing ABI_2021_May...
Results saved to ./cleaned_weather_data/ABI_2021_May.csv
Processing ABI_2022_Aug...
Results saved to ./cleaned_weather_data/ABI_2022_Aug.csv
Processing ABI_2022_Jul...
Results saved to ./cleaned_weather_data/ABI_2022_Jul.csv
Processing ABI_2022_Jun...
Results saved to ./cleaned_weather_data/ABI_2022_Jun.csv
Processing ABI_2022_May...
Results saved to ./cleaned_weather_data/ABI_2022_May.csv
Processing ABI_2023_Aug...
Results saved to ./cleaned_weather_data/ABI_2023_Aug.csv
Processing ABI_2023_Jul...
Results saved to ./cleaned_weather_data/ABI_2023_Jul.csv
Processing ABI_2023_Jun...
Results saved to ./cleaned_weather_data/ABI_2023_Jun.csv
Processing ABI_2023_May...

In [9]:
import pandas as pd
import itertools
import os
import re
from datetime import datetime

# Define data directory
data_dir = "./cleaned_data/"

# Get all data files
data_files = [
    'May2021.csv', 'May2022.csv', 'May2023.csv', 'May2024.csv',
    'June2021.csv', 'June2022.csv', 'June2023.csv', 'June2024.csv',
    'July2021.csv', 'July2022.csv', 'July2023.csv', 'July2024.csv',
    'August2021.csv', 'August2022.csv', 'August2023.csv', 'August2024.csv'
]

# Convert to full paths
data_files_with_path = [os.path.join(data_dir, file) for file in data_files]

# Sort files by date (newest to oldest)
def extract_date(filepath):
    filename = os.path.basename(filepath)
    match = re.match(r'(\w+)(\d{4})\.csv', filename)
    if match:
        month, year = match.groups()
        month_map = {
            'January': 1, 'February': 2, 'March': 3, 'April': 4,
            'May': 5, 'June': 6, 'July': 7, 'August': 8,
            'September': 9, 'October': 10, 'November': 11, 'December': 12
        }
        month_num = month_map.get(month, 0)
        return datetime(int(year), month_num, 1)
    return datetime(1900, 1, 1)  # Default date

# Sort from newest to oldest
sorted_data_files = sorted(data_files_with_path, key=extract_date, reverse=True)

# Read top 30 airports data
top_airports = pd.read_csv('top_100_airports.csv')
top_30_airport_codes = top_airports.head(30)['ORIGIN_IATA'].tolist()

# Create all possible airport pair combinations
airport_pairs = list(itertools.combinations(top_30_airport_codes, 2))

# Create dictionaries to store results
results = {}
data_sources = {}

# Pre-load all data files for efficiency
distance_data_frames = {}
for file in sorted_data_files:
    if os.path.exists(file):
        try:
            distance_data_frames[file] = pd.read_csv(file)
            print(f"Loaded: {file}")
        except Exception as e:
            print(f"Failed to load {file}: {e}")
    else:
        print(f"Warning: File {file} does not exist and will be skipped.")

print(f"Loaded {len(distance_data_frames)} data files.")
print(f"There are {len(airport_pairs)} airport pairs to query.")

# Create progress counter
count = 0
total = len(airport_pairs)

# Iterate through all airport pairs
for origin, dest in airport_pairs:
    count += 1
    if count % 50 == 0:
        print(f"Progress: {count}/{total} ({(count/total*100):.1f}%)")

    found = False

    # Search in each file by priority
    for file, df in distance_data_frames.items():
        # Look for distance from origin to dest
        query1 = df[(df['ORIGIN_IATA'] == origin) & (df['DEST_IATA'] == dest)]

        # Look for distance from dest to origin (reverse)
        query2 = df[(df['ORIGIN_IATA'] == dest) & (df['DEST_IATA'] == origin)]

        # If distance data is found
        if not query1.empty:
            distance = query1['DISTANCE'].values[0]
            results[(origin, dest)] = distance
            data_sources[(origin, dest)] = os.path.basename(file)  # Store filename only
            found = True
            break
        elif not query2.empty:
            distance = query2['DISTANCE'].values[0]
            results[(origin, dest)] = distance
            data_sources[(origin, dest)] = os.path.basename(file)  # Store filename only
            found = True
            break

    # If no data found in any file
    if not found:
        results[(origin, dest)] = None
        data_sources[(origin, dest)] = 'Not Found'

# Convert results to DataFrame
result_df = pd.DataFrame([
    {'Origin': pair[0], 'Destination': pair[1], 'Distance': distance, 'Data_Source': data_sources[pair]}
    for pair, distance in results.items()
])

# Save results
output_path = os.path.join(data_dir, 'top30_airport_distances.csv')
result_df.to_csv(output_path, index=False)
print(f"Results saved to: {output_path}")

# Output statistics
total_pairs = len(airport_pairs)
pairs_with_distance = len([d for d in results.values() if d is not None])

print("\nResult Statistics:")
print(f"Total airport pairs: {total_pairs}")
print(f"Pairs with distance data: {pairs_with_distance} ({pairs_with_distance/total_pairs*100:.1f}%)")
print(f"Pairs missing distance data: {total_pairs - pairs_with_distance} ({(total_pairs-pairs_with_distance)/total_pairs*100:.1f}%)")

# Statistics by data source
source_stats = result_df['Data_Source'].value_counts().to_dict()
print("\nData Source Statistics:")
for source, count in sorted(source_stats.items()):
    print(f"Data from {source}: {count} entries ({count/total_pairs*100:.1f}%)")

Loaded: ./cleaned_data/August2024.csv
Loaded: ./cleaned_data/July2024.csv
Loaded: ./cleaned_data/June2024.csv
Loaded: ./cleaned_data/May2024.csv
Loaded: ./cleaned_data/August2023.csv


C:\Users\Administrator\AppData\Local\Temp\ipykernel_25296\1701520018.py:55: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  distance_data_frames[file] = pd.read_csv(file)


Loaded: ./cleaned_data/July2023.csv
Loaded: ./cleaned_data/June2023.csv
Loaded: ./cleaned_data/May2023.csv
Loaded: ./cleaned_data/August2022.csv
Loaded: ./cleaned_data/July2022.csv
Loaded: ./cleaned_data/June2022.csv
Loaded: ./cleaned_data/May2022.csv
Loaded: ./cleaned_data/August2021.csv
Loaded: ./cleaned_data/July2021.csv
Loaded: ./cleaned_data/June2021.csv
Loaded: ./cleaned_data/May2021.csv
Loaded 16 data files.
There are 435 airport pairs to query.
Progress: 50/435 (11.5%)
Progress: 100/435 (23.0%)
Progress: 150/435 (34.5%)
Progress: 200/435 (46.0%)
Progress: 250/435 (57.5%)
Progress: 300/435 (69.0%)
Progress: 350/435 (80.5%)
Progress: 400/435 (92.0%)
Results saved to: ./cleaned_data/top30_airport_distances.csv

Result Statistics:
Total airport pairs: 435
Pairs with distance data: 417 (95.9%)
Pairs missing distance data: 18 (4.1%)

Data Source Statistics:
Data from August2023.csv: 2 entries (0.5%)
Data from August2024.csv: 410 entries (94.3%)
Data from July2024.csv: 2 entries (0.5%